In [252]:
#-*-coding: utf-8-*-
from random import choice
##############################################################################
# Variables globales
##############################################################################

# Crea los conectivos
conectivos = ['Y', 'O', '>', '=']
# Crea las letras minúsculas a-z
letrasProposicionales = [chr(x) for x in range(97, 123)]
# inicializa la lista de interpretaciones
listaInterpsVerdaderas = []
# inicializa la lista de hojas
listaHojas = []

##############################################################################
# Definición de objeto tree y funciones de árboles
##############################################################################

class Tree(object):
	def __init__(self, label, left, right):
		self.left = left
		self.right = right
		self.label = label

def Inorder(f):
    # Imprime una formula como cadena dada una formula como arbol
    # Input: tree, que es una formula de logica proposicional
    # Output: string de la formula
	if f.right == None:
		return f.label
	elif f.label == '-':
		return f.label + Inorder(f.right)
	else:
		return "(" + Inorder(f.left) + f.label + Inorder(f.right) + ")"

def String2Tree(A):
	# Crea una formula como tree dada una formula como cadena escrita en notacion polaca inversa
	# Input: - A, lista de caracteres con una formula escrita en notacion polaca inversa
	#        - letrasProposicionales, lista de letras proposicionales
	#        - conectivos, lista de conectivos
	# Output: formula como tree
	pila = []
	for c in A:
		# print("Examinando " + str(c))
		if c in letrasProposicionales:
			# print(u"El símbolo es letra proposicional")
			pila.append(Tree(c, None, None))
		elif c == '-':
			# print("Negamos")
			formulaAux = Tree(c, None, pila[-1])
			del pila[-1]
			pila.append(formulaAux)
		elif c in conectivos:
			# print("Unimos mediante conectivo")
			formulaAux = Tree(c, pila[-1], pila[-2])
			del pila[-1]
			del pila[-1]
			pila.append(formulaAux)
		else:
			print(u"Hay un problema: el símbolo " + str(c) + " no se reconoce")
	return pila[-1]

def Inorder2Tree(A):
	if len(A) == 1:
		return Tree(A[0], None, None)
	elif A[0] == '-':
		return Tree(A[0], None, Inorder2Tree(A[1:]))
	elif A[0] == "(":
		counter = 0 #Contador de parentesis
		for i in range(1, len(A)):
			if A[i] == "(":
				counter += 1
			elif A[i] == ")":
				counter -=1
			elif (A[i] in ['Y', 'O', '>', '=']) and (counter == 0):
				return Tree(A[i], Inorder2Tree(A[1:i]), Inorder2Tree(A[i + 1:-1]))
	else:
		return -1

def list2String(l):
	ls = []
	for f in l:
		ls.append(Inorder(f))
	return ls

##############################################################################
# Definición de funciones de tableaux
##############################################################################

def imprime_hoja(H):
	cadena = "{"
	primero = True
	for f in H:
		if primero == True:
			primero = False
		else:
			cadena += ", "
		cadena += Inorder(f)
	return cadena + "}"

def imprime_listaHojas(L):
	for h in L:
		print(imprime_hoja(h))

def complemento(l):
	# Esta función devuelve el complemento de un literal
	# Input: l, un literal
	# Output: x, un literal
	if l.label == "-":
		return l.right
	else:
		return Tree("-", None, l)

def par_complementario(l):
    # Esta función determina si una lista de solo literales
    # contiene un par complementario
    # Input: l, una lista de literales
    # Output: True/False
    ls = []
    lsc = []
    for x in l:
        ls.append(Inorder(x))
        lsc.append(Inorder(complemento(x)))
    for c in lsc:
        if c in ls:
            return True
    return False

def es_literal(f):
    # Esta función determina si el árbol f es un literal
    # Input: f, una fórmula como árbol
    # Output: True/False
	if f.right == None:
		return True
	else:
		if f.label == "-" and f.right.right == None:
			return True
	return False

def no_literales(l):
    # Esta función determina si una lista de fórmulas contiene
    # solo literales
    # Input: l, una lista de fórmulas como árboles
    # Output: None/f, tal que f no es literal\
	for f in l:
		if not(es_literal(f)):
			return f
	return None

def clasificacion(f):
	# clasifica una fórmula como alfa o beta
	# Input: f, una fórmula como árbol
	# Output: string de la clasificación de la formula
	if f.label =='-':
		if f.right.label == '-':
			return 'Alfa1'
		elif f.right.label == 'O':
			return 'Alfa3'
		elif f.right.label == '>':
			return 'Alfa4'
		elif f.right.label == 'Y':
			return 'Beta3'
	elif f.label == 'Y':
		return 'Alfa2'
	elif f.label == '=':
		return 'Alfa5'
	elif f.label == 'O':
		return 'Beta1'
	elif f.label == '>':
		return 'Beta2'

def clasificacion(f):
	# clasifica una fórmula como alfa o beta
	# Input: f, una fórmula como árbol
	# Output: string de la clasificación de la formula
	if f.label =='-':
		if f.right.label == '-':
			return 'Alfa1'
		elif f.right.label == 'O':
			return 'Alfa3'
		elif f.right.label == '>':
			return 'Alfa4'
		elif f.right.label == 'Y':
			return 'Beta3'
	elif f.label == 'Y':
		return 'Alfa2'
	elif f.label == '=':
		return 'Alfa5'
	elif f.label == 'O':
		return 'Beta1'
	elif f.label == '>':
		return 'Beta2'

def clasifica_y_extiende(f, h):
	# Extiende listaHojas de acuerdo a la regla respectiva
	# Input: f, una fórmula como árbol
	# 		 h, una hoja (lista de fórmulas como árboles)
	# Output: no tiene output, pues modifica la variable global listaHojas

	global listaHojas

	print("Formula:", Inorder(f))
	print("Hoja:", imprime_hoja(h))

	assert(f in h), "La formula no esta en la lista!"

	clase = clasificacion(f)
	print("Clasificada como:", clase)
	assert(clase != None), "Formula incorrecta " + imprime_hoja(h)

	if clase == 'Alfa1':
		aux = [x for x in h if x != f] + [f.right.right]
		listaHojas.remove(h)
		listaHojas.append(aux)
	elif clase == 'Alfa2':
		aux = [x for x in h if x != f] + [f.right] + [f.left]
		listaHojas.remove(h)
		listaHojas.append(aux)
	elif clase == 'Alfa3':
		aux = [x for x in h if x != f] + [Tree('-',None,f.right.left)] + [Tree('-',None,f.right.right)]
		listaHojas.remove(h)
		listaHojas.append(aux)
	elif clase == 'Alfa4':
		aux = [x for x in h if x != f] + [f.right.left] + [Tree('-',None,f.right.right)]
		listaHojas.remove(h)
		listaHojas.append(aux)
	elif clase == 'Alfa5':
		aux = [x for x in h if x != f] + [Tree('>',f.left,f.right)] + [Tree('>',f.right,f.left)]
		listaHojas.remove(h)
		listaHojas.append(aux)
	elif clase == 'Beta1':
		listaHojas.remove(h)
		aux = [x for x in h if x != f] + [f.right]
		listaHojas.append(aux)
		aux = [x for x in h if x != f] + [f.left]
		listaHojas.append(aux)
		pass
	elif clase == 'Beta2':
		listaHojas.remove(h)
		aux = [x for x in h if x != f] + [Tree('-',None,f.right.left)]
		listaHojas.append(aux)
		aux = [x for x in h if x != f] + [f.right.right]
		listaHojas.append(aux)
	elif clase == 'Beta3':
		listaHojas.remove(h)
		aux = [x for x in h if x != f] + [Tree('-',None,f.right.left)]
		listaHojas.append(aux)
		aux = [x for x in h if x != f] + [Tree('-',None,f.right.right)]
		listaHojas.append(aux)

def Tableaux(f):

	# Algoritmo de creacion de tableau a partir de lista_hojas
	# Imput: - f, una fórmula como string en notación polaca inversa
	# Output: interpretaciones: lista de listas de literales que hacen
	#		 verdadera a f

	global listaHojas
	global listaInterpsVerdaderas

	A = String2Tree(f)
	print(u'La fórmula introducida es:\n', Inorder(A))

	listaHojas = [[A]]

	while (len(listaHojas) > 0):
		h = choice(listaHojas)
		print("Trabajando con hoja:\n", imprime_hoja(h))
		x = no_literales(h)
		if x == None:
			if par_complementario(h):
				listaHojas.remove(h)
			else:
				listaInterpsVerdaderas.append(h)
				listaHojas.remove(h)
		else:
			clasifica_y_extiende(x, h)

	return listaInterpsVerdaderas

In [253]:
par_complementario([Tree('1',None,None), Tree('2',None,None), Tree('-',None,Tree('3',None,None)), Tree('1',None,None)])

False

In [254]:
par_complementario([Tree('b',None,None), Tree('-',None,Tree('a',None,None)), Tree('-',None,Tree('c',None,None)), Tree('a',None,None), Tree('d',None,None)])

True

In [255]:
par_complementario([Tree('-',None,Tree('Z1',None,None)), Tree('S1',None,None), Tree('-',None,Tree('S10',None,None)), Tree('Z10',None,None)])

False

In [256]:
par_complementario([Tree('-',None,Tree('q',None,None)), Tree('-',None,Tree('p',None,None)), Tree('q',None,None), Tree('-',None,Tree('r',None,None))])

True

In [257]:
print(list2String([Tree('-',None,Tree('p',None,None)),Tree('q',None,None),Tree('-',None,Tree('p',None,None)),Tree('r',None,None),Tree('-',None,Tree('q',None,None)),Tree('p',None,None)]))
no_literales([Tree('-',None,Tree('p',None,None)),Tree('q',None,None),Tree('-',None,Tree('p',None,None)),Tree('r',None,None),Tree('-',None,Tree('q',None,None)),Tree('p',None,None)])

['-p', 'q', '-p', 'r', '-q', 'p']


In [258]:
print(list2String([Tree('q',None,None),Tree('-',None,Tree('p',None,None)),Tree('-',None,Tree('-',None,Tree('p',None,None))),Tree('-',None,Tree('q',None,None))]))
no_literales([Tree('q',None,None),Tree('-',None,Tree('p',None,None)),Tree('-',None,Tree('-',None,Tree('p',None,None))),Tree('-',None,Tree('q',None,None))])

['q', '-p', '--p', '-q']


In [259]:
#-*-coding: utf-8-*-
from random import choice
##############################################################################
# Variables globales
##############################################################################

# Crea los conectivos
conectivos = ['Y', 'O', '>', '=']
# Crea las letras minúsculas a-z
letrasProposicionales = [chr(x) for x in range(97, 123)]
# inicializa la lista de interpretaciones
listaInterpsVerdaderas = []
# inicializa la lista de hojas
listaHojas = []

##############################################################################
# Definición de objeto tree y funciones de árboles
##############################################################################

class Tree(object):
	def __init__(self, label, left, right):
		self.left = left
		self.right = right
		self.label = label

def Inorder(f):
    # Imprime una formula como cadena dada una formula como arbol
    # Input: tree, que es una formula de logica proposicional
    # Output: string de la formula
	"""
	if type(f) == str:
		return f
	"""
	if f.right == None:
		return f.label
	elif f.label == '-':
		return f.label + Inorder(f.right)
	else:
		return "(" + Inorder(f.left) + f.label + Inorder(f.right) + ")"

def String2Tree(A):
	# Crea una formula como tree dada una formula como cadena escrita en notacion polaca inversa
	# Input: - A, lista de caracteres con una formula escrita en notacion polaca inversa
	#        - letrasProposicionales, lista de letras proposicionales
	#        - conectivos, lista de conectivos
	# Output: formula como tree
	pila = []
	for c in A:
		# print("Examinando " + str(c))
		if c in letrasProposicionales:
			# print(u"El símbolo es letra proposicional")
			pila.append(Tree(c, None, None))
		elif c == '-':
			# print("Negamos")
			formulaAux = Tree(c, None, pila[-1])
			del pila[-1]
			pila.append(formulaAux)
		elif c in conectivos:
			# print("Unimos mediante conectivo")
			formulaAux = Tree(c, pila[-1], pila[-2])
			del pila[-1]
			del pila[-1]
			pila.append(formulaAux)
		else:
			print(u"Hay un problema: el símbolo " + str(c) + " no se reconoce")
	return pila[-1]

def Inorder2Tree(A):
	if len(A) == 1:
		return Tree(A[0], None, None)
	elif A[0] == '-':
		return Tree(A[0], None, Inorder2Tree(A[1:]))
	elif A[0] == "(":
		counter = 0 #Contador de parentesis
		for i in range(1, len(A)):
			if A[i] == "(":
				counter += 1
			elif A[i] == ")":
				counter -=1
			elif (A[i] in ['Y', 'O', '>', '=']) and (counter == 0):
				return Tree(A[i], Inorder2Tree(A[1:i]), Inorder2Tree(A[i + 1:-1]))
	else:
		return -1

##############################################################################
# Definición de funciones de tableaux
##############################################################################

def imprime_hoja(H):
	cadena = "{"
	primero = True
	for f in H:
		if primero == True:
			primero = False
		else:
			cadena += ", "
		cadena += Inorder(f)
	return cadena + "}"

def imprime_listaHojas(L):
	for h in L:
		print(imprime_hoja(h))

def complemento(l):
	# Esta función devuelve el complemento de un literal
	# Input: l, un literal
	# Output: x, un literal
	if l.label == "-":
		return l.right
	else:
		return Tree("-", None, l)

def par_complementario(l):
    # Esta función determina si una lista de solo literales
    # contiene un par complementario
    # Input: l, una lista de literales
    # Output: True/False
    ls = []
    lsc = []
    for x in l:
        ls.append(Inorder(x))
        lsc.append(Inorder(complemento(x)))
    for c in lsc:
        if c in ls:
            return True
    return False

def es_literal(f):
    # Esta función determina si el árbol f es un literal
    # Input: f, una fórmula como árbol
    # Output: True/False
	if f.right == None:
		return True
	else:
		if f.label == "-" and f.right.right == None:
			return True
	return False

def no_literales(l):
    # Esta función determina si una lista de fórmulas contiene
    # solo literales
    # Input: l, una lista de fórmulas como árboles
    # Output: None/f, tal que f no es literal\
	for f in l:
		if not(es_literal(f)):
			return f
	return None

def clasificacion(f):
	# clasifica una fórmula como alfa o beta
	# Input: f, una fórmula como árbol
	# Output: string de la clasificación de la formula
	if f.label =='-':
		if f.right.label == '-':
			return 'Alfa1'
		elif f.right.label == 'O':
			return 'Alfa3'
		elif f.right.label == '>':
			return 'Alfa4'
		elif f.right.label == 'Y':
			return 'Beta1'
	elif f.label == 'Y':
		return 'Alfa2'
	elif f.label == '=':
		return 'Alfa5'
	elif f.label == 'O':
		return 'Beta2'
	elif f.label == '>':
		return 'Beta3'

def clasifica_y_extiende(f, h):
	# Extiende listaHojas de acuerdo a la regla respectiva
	# Input: f, una fórmula como árbol
	# 		 h, una hoja (lista de fórmulas como árboles)
	# Output: no tiene output, pues modifica la variable global listaHojas

	global listaHojas

	print("Formula:", Inorder(f))
	print("Hoja:", imprime_hoja(h))

	assert(f in h), "La formula no esta en la lista!"

	clase = clasificacion(f)
	print("Clasificada como:", clase)
	assert(clase != None), "Formula incorrecta " + imprime_hoja(h)

	if clase == 'Alfa1':
		aux = [x for x in h if x != f] + [f.right.right]
		listaHojas.remove(h)
		listaHojas.append(aux)
	elif clase == 'Alfa2':
		aux = [x for x in h if x != f] + [f.right] + [f.left]
		listaHojas.remove(h)
		listaHojas.append(aux)
	elif clase == 'Alfa3':
		aux = [x for x in h if x != f] + [Tree('-',None,f.right.left)] + [Tree('-',None,f.right.right)]
		listaHojas.remove(h)
		listaHojas.append(aux)
	elif clase == 'Alfa4':
		aux = [x for x in h if x != f] + [f.right.left] + [Tree('-',None,f.right.right)]
		listaHojas.remove(h)
		listaHojas.append(aux)
	elif clase == 'Alfa5':
		aux = [x for x in h if x != f] + [Tree('>',f.left,f.right)] + [Tree('>',f.right,f.left)]
		listaHojas.remove(h)
		listaHojas.append(aux)
	elif clase == 'Beta2':
		aux = [x for x in h if x != f] + [f.right]
		aux2 = [x for x in h if x != f] + [f.left]
		listaHojas.remove(h)
		listaHojas.append(aux)
		listaHojas.append(aux2)
	elif clase == 'Beta3':
		aux = [x for x in h if x != f] + [Tree('-',None,f.right.left)]
		aux2 = [x for x in h if x != f] + [f.right.right]
		listaHojas.remove(h)
		listaHojas.append(aux)
		listaHojas.append(aux2)
	elif clase == 'Beta1':
		aux = [x for x in h if x != f] + [Tree('-',None,f.right.left)]
		aux2 = [x for x in h if x != f] + [Tree('-',None,f.right.right)]
		listaHojas.remove(h)
		listaHojas.append(aux)
		listaHojas.append(aux2)

def Tableaux(f):

	# Algoritmo de creacion de tableau a partir de lista_hojas
	# Imput: - f, una fórmula como string en notación polaca inversa
	# Output: interpretaciones: lista de listas de literales que hacen
	#		 verdadera a f

	global listaHojas
	global listaInterpsVerdaderas

	A = String2Tree(f)
	print(u'La formula introducida es:\n', Inorder(A))

	listaHojas = [[A]]

	while (len(listaHojas) > 0):
		h = choice(listaHojas)
		print("Trabajando con hoja:\n", imprime_hoja(h))
		x = no_literales(h)
		if x == None:
			if par_complementario(h):
				listaHojas.remove(h)
			else:
				listaInterpsVerdaderas.append(h)
				listaHojas.remove(h)
		else:
			clasifica_y_extiende(x, h)

	return listaInterpsVerdaderas

In [260]:
print(list2String([Tree('p',None,None),Tree('q',None,None),Tree('O',Tree('p',None,None),Tree('q',None,None)),Tree('-',None,Tree('q',None,None)),Tree('-',None,Tree('p',None,None))]))
no_literales([Tree('p',None,None),Tree('q',None,None),Tree('O',Tree('p',None,None),Tree('q',None,None)),Tree('-',None,Tree('q',None,None)),Tree('-',None,Tree('p',None,None))])

['p', 'q', '(pOq)', '-q', '-p']


In [261]:
print(Inorder(Tree('-',None,Tree('Y',Tree('p',None,None),Tree('q',None,None)))))
es_literal(Tree('-',None,Tree('Y',Tree('p',None,None),Tree('q',None,None))))

-(pYq)


False

In [262]:
print(Inorder(Tree('alpha',None,None)))
es_literal(Tree('alpha',None,None))

alpha


True

In [263]:
print(Inorder(Tree('O',Tree('k',None,None),Tree('Y',Tree('i',None,None),Tree('j',None,None)))))
es_literal(Tree('O',Tree('k',None,None),Tree('Y',Tree('i',None,None),Tree('j',None,None))))

(kO(iYj))


False

In [264]:
print(Inorder(Tree('-',None,Tree('p',None,None))))
es_literal(Tree('-',None,Tree('p',None,None)))

-p


True

In [265]:
clasificacion(Tree('O',Tree('-',None,Tree('s',None,None)),Tree('Y',Tree('t',None,None),Tree('>',Tree('u',None,None),Tree('v',None,None)))))

'Beta2'

In [266]:
f = Inorder2Tree('-(-(r>s)Oq)')
h = [f, Inorder2Tree('-p')]
listaHojas = [h]
clasifica_y_extiende(f, h)
imprime_listaHojas(listaHojas)

Formula: -(-(r>s)Oq)
Hoja: {-(-(r>s)Oq), -p}
Clasificada como: Alfa3
{-p, --(r>s), -q}


In [267]:
f = Inorder2Tree('-(pO(rYs))')
h = [f, Inorder2Tree('q'), Inorder2Tree('p')]
listaHojas = [h]
clasifica_y_extiende(f, h)
imprime_listaHojas(listaHojas)

Formula: -(pO(rYs))
Hoja: {-(pO(rYs)), q, p}
Clasificada como: Alfa3
{q, p, -p, -(rYs)}


In [268]:
f = Inorder2Tree('-(rYs)')
h = [f, Inorder2Tree('-p')]
listaHojas = [h]
clasifica_y_extiende(f, h)
imprime_listaHojas(listaHojas)

Formula: -(rYs)
Hoja: {-(rYs), -p}
Clasificada como: Beta1
{-p, -r}
{-p, -s}


In [269]:
f = Inorder2Tree('(pYq)')
h = [f, Inorder2Tree('-q')]
listaHojas = [h]
clasifica_y_extiende(f, h)
imprime_listaHojas(listaHojas)

Formula: (pYq)
Hoja: {(pYq), -q}
Clasificada como: Alfa2
{-q, q, p}


In [270]:
Tableaux('qpOq-p-YY')

La formula introducida es:
 ((-pY-q)Y(pOq))
Trabajando con hoja:
 {((-pY-q)Y(pOq))}
Formula: ((-pY-q)Y(pOq))
Hoja: {((-pY-q)Y(pOq))}
Clasificada como: Alfa2
Trabajando con hoja:
 {(pOq), (-pY-q)}
Formula: (pOq)
Hoja: {(pOq), (-pY-q)}
Clasificada como: Beta2
Trabajando con hoja:
 {(-pY-q), p}
Formula: (-pY-q)
Hoja: {(-pY-q), p}
Clasificada como: Alfa2
Trabajando con hoja:
 {p, -q, -p}
Trabajando con hoja:
 {(-pY-q), q}
Formula: (-pY-q)
Hoja: {(-pY-q), q}
Clasificada como: Alfa2
Trabajando con hoja:
 {q, -q, -p}


[]